In [ ]:
import os
import json
import codecs
import pandas as pd
import numpy as np
import random

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import metrics

In [ ]:
import random

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Change working directory to be current folder
os.chdir('/content/gdrive/My Drive/MyColab/PLP')
!ls

Mounted at /content/gdrive
 bigram_vectorizer.pkl		   Small_talk_Intent.csv
 ch21.pkl			   Small_talk_Intent.xlsx
 chatbot_intent			  'spi_index_labelled vFinal.csv'
 clr_svm.pkl			  'spi_index_labelled vFinal.db'
 corpus_embeddings.pkl		   terms_questions.csv
 GQLquestion.pkl		   textsql.csv
 investopedia_corpus.csv	   tfidf_matrix.pkl
 investopedia_terms_question.csv   tfidf.pkl
 SBert_embedder.pkl


In [ ]:
investopedia_corpus = pd.read_csv('investopedia_corpus.csv')
investopedia_terms = list(investopedia_corpus['term'].map(lambda x: x.lower()))

In [ ]:
#prepare the questions for investopedia terms
import random
import csv
question_tags = ["what is",
"what do you mean by",
"explain",
"define",
"can you tell me about",
"what is meant by",
"can you give me an idea about",
"can you please give short description about",
"briefly explain",
"what do you know about",
"please explain",
"can you say about",
"what is meant by",
"describe",
"what do you know",
"help me with"]
terms_question_corpus = []
for terms in investopedia_terms:
  tempQuestionTag = question_tags.copy()
  for question_tag in question_tags:
    terms_question_corpus.append([terms,f'{question_tag} {terms}'])

with open('terms_questions.csv', 'w') as f:
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(['terms', 'questions'])
    write.writerows(terms_question_corpus)

In [ ]:
investopedia_terms_question = pd.read_csv('terms_questions.csv')
investopedia_terms_question.head()

,terms,questions
0,abandonment clause,what is abandonment clause
1,abandonment clause,what do you mean by abandonment clause
2,abandonment clause,explain abandonment clause
3,abandonment clause,define abandonment clause
4,abandonment clause,can you tell me about abandonment clause


In [ ]:
#use investopedia_terms_question as the question bank corpus
questions_corpus = list(investopedia_terms_question['questions'])
questions_corpus[:5]

['what is abandonment clause',
 'what do you mean by abandonment clause',
 'explain abandonment clause',
 'define abandonment clause',
 'can you tell me about abandonment clause']

In [ ]:
#apply TFIDFVectorizer to check the sentence similarity
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')
tfidf_matrix = tfidf.fit_transform(questions_corpus)

In [ ]:
query = 'please let me what is abandonment and salvage'
query_vector = tfidf.transform([query.lower()])
cosine_sim = cosine_similarity(tfidf_matrix, query_vector)

In [ ]:
#retrieve the top 5 simiar sentence with respect to the query
scores_with_corpus_index = np.concatenate((cosine_sim,np.array([index for index,value in enumerate(questions_corpus)]).reshape(-1,1)),axis=1)
sim_scores = sorted(scores_with_corpus_index, key=lambda x: x[0], reverse=True)
sim_scores[:5]

[array([ 1., 16.]),
 array([ 1., 29.]),
 array([ 0.97043579, 18.        ]),
 array([ 0.97043579, 26.        ]),
 array([ 0.96134585, 21.        ])]

In [ ]:
investopedia_terms_question.iloc[2,:]

terms                abandonment clause
questions    explain abandonment clause
Name: 2, dtype: object

In [ ]:
#retrieve the top 5 similar sentence and terms
for data in sim_scores[:5]:
  corpus_index = int(data[1])
  print(data[0])
  print(investopedia_terms_question.iloc[corpus_index,:])
  print()

1.0
terms                abandonment and salvage
questions    what is abandonment and salvage
Name: 16, dtype: object

1.0
terms                 abandonment and salvage
questions    describe abandonment and salvage
Name: 29, dtype: object

0.9704357931495713
terms                abandonment and salvage
questions    explain abandonment and salvage
Name: 18, dtype: object

0.9704357931495713
terms                       abandonment and salvage
questions    please explain abandonment and salvage
Name: 26, dtype: object

0.9613458535982483
terms                         abandonment and salvage
questions    what is meant by abandonment and salvage
Name: 21, dtype: object



In [ ]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.0 MB/s 
     |████████████████████████████████| 5.3 MB 15.8 MB/s 
     |████████████████████████████████| 1.3 MB 31.8 MB/s 
     |████████████████████████████████| 163 kB 37.7 MB/s 
     |████████████████████████████████| 7.6 MB 39.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=a950fb8a2f93c3100b67075edaea6d7d9a552dedcab2d9d50e7950a401db4e55
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [ ]:
!pip list transformer

In [ ]:
#apply Sentence Bert Transformer to extract the sentence simililarity
from sentence_transformers import SentenceTransformer
from sentence_transformers import util

#encode the question corpus by using sentence bert transformer
embedder = SentenceTransformer('msmarco-distilbert-base-v4')
corpus_embeddings = embedder.encode(questions_corpus, convert_to_tensor=True)

#apply cosine similarity to compare the question corpus and the new query
query = query = 'please tell me what is abandonment and salvage'
query_embedding = embedder.encode([query], convert_to_tensor=True)
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0].cpu()

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/319 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
import torch

scores,indices = torch.topk(cos_scores, k=5)

#retrieve the top 5 similar sentence and terms
for index, corpusIndex in enumerate(indices):
  print(scores[index])
  print(investopedia_terms_question.iloc[int(corpusIndex.item()),:])
  print()

tensor(0.9275)
terms                               abandonment and salvage
questions    what do you know about abandonment and salvage
Name: 25, dtype: object

tensor(0.9202)
terms                       abandonment and salvage
questions    please explain abandonment and salvage
Name: 26, dtype: object

tensor(0.9166)
terms                         abandonment and salvage
questions    what do you know abandonment and salvage
Name: 30, dtype: object

tensor(0.9158)
terms                              abandonment and salvage
questions    can you tell me about abandonment and salvage
Name: 20, dtype: object

tensor(0.9146)
terms                abandonment and salvage
questions    what is abandonment and salvage
Name: 16, dtype: object



In [ ]:
tfidf_score = torch.FloatTensor(np.transpose(cosine_sim)[0])
k_coeff = 0.5
combined_scores = k_coeff*cos_scores + (1-k_coeff)*tfidf_score

import torch

scores,indices = torch.topk(combined_scores, k=5)

#retrieve the top 5 similar sentence and terms
for index, corpusIndex in enumerate(indices):
  print(scores[index])
  print(investopedia_terms_question.iloc[int(corpusIndex.item()),:])
  print()

tensor(0.9573)
terms                abandonment and salvage
questions    what is abandonment and salvage
Name: 16, dtype: object

tensor(0.9454)
terms                 abandonment and salvage
questions    describe abandonment and salvage
Name: 29, dtype: object

tensor(0.9453)
terms                       abandonment and salvage
questions    please explain abandonment and salvage
Name: 26, dtype: object

tensor(0.9444)
terms                               abandonment and salvage
questions    what do you know about abandonment and salvage
Name: 25, dtype: object

tensor(0.9390)
terms                         abandonment and salvage
questions    what do you know abandonment and salvage
Name: 30, dtype: object



In [ ]:
import pickle
with open('SBert_embedder.pkl', 'wb') as f:
  pickle.dump(embedder,f)
with open('corpus_embeddings.pkl', 'wb') as f:
  pickle.dump(corpus_embeddings,f)
with open('tfidf.pkl', 'wb') as f:
  pickle.dump(tfidf,f)
with open('tfidf_matrix.pkl', 'wb') as f:
  pickle.dump(tfidf_matrix,f)